##Chapter 16: Production Infrastructure Part 2: Feature Store
Author: Ben Wilson

#### Summary
In this notebook, we'll be looking at leveraging the Databricks Feature Store service to greatly simplify and decouple feature engineering tasks from modeling tasks, promoting feature reuse and visibility throughout an organization. </br>

#### Notes
At the current time of writing this, this API for Feature Store service is undergoing active development. The intention of showcasing this functionality is to illustrate the concept of feature stores and what they can bring to ML production work, rather than focusing specifically on this implementation of a feature store.

In [0]:
%pip install catboost

Python interpreter will be restarted.
Collecting catboost
 Downloading catboost-0.26-cp38-none-manylinux1_x86_64.whl (69.2 MB)
Requirement already satisfied: six in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from catboost) (1.15.0)
Requirement already satisfied: scipy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from catboost) (1.5.2)
Requirement already satisfied: plotly in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from catboost) (4.14.3)
Collecting graphviz
 Downloading graphviz-0.16-py2.py3-none-any.whl (19 kB)
Requirement already satisfied: matplotlib in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from catboost) (3.2.2)
Requirement already satisfied: pandas>=0.24.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from catboost) (1.1.5)
Requirement already satisfied: numpy>=1.16.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from catboost) (1.19.2)
Requirement already satisfied: retrying>=1.3.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from plotly->catboost) (1.3.3)
Requirement already satisfied: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from matplotlib->catboost) (2.4.7)
Requirement already satisfied: cycler>=0.10 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from matplotlib->catboost) (0.10.0)
Requirement already satisfied: kiwisolver>=1.0.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from matplotlib->catboost) (1.3.0)
Requirement already satisfied: python-dateutil>=2.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from matplotlib->catboost) (2.8.1)
Requirement already satisfied: pytz>=2017.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-eb92f591-1510-4d89-a4ab-47dab10572c4/lib/python3.8/site-packages (from pandas>=0.24.0->catboost) (2020.5)
Installing collected packages: graphviz, catboost
Successfully installed catboost-0.26 graphviz-0.16
Python interpreter will be restarted.

In [0]:
import requests
from dataclasses import dataclass
from typing import List

from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import when

from databricks import feature_store
from databricks.feature_store import FeatureLookup
import mlflow
from mlflow.tracking.client import MlflowClient

from catboost import CatBoostClassifier, metrics as cb_metrics

from sklearn import metrics
from sklearn.model_selection import train_test_split


Data set acquisition and table registration

In [0]:
class DataIngest:
  
  def __init__(self, url, local, source, sink, schema, database, table_name):
    self.url = url
    self.local = local
    self.source = source
    self.sink = sink
    self.schema = schema
    self.database = database
    self.table_name = table_name
    
  def _acquire_raw(self):
    response = requests.get(self.url, stream=True)
    with open(self.url.split("/")[-1], "wb") as data:
      data.write(response.content)
    response.close()
    
  def _transfer_local(self):
    dbutils.fs.mv(self.local, self.source)
    
  def _read_source(self):
    return spark.read.csv(self.source, header=True, inferSchema=False, schema=self.schema)
  
  def _write_source(self, data):
    data.write.format("delta").mode("overwrite").option("mergeSchema", "true").option("overwriteSchema", "true").save(self.sink)
    
  def _create_table(self):
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {self.database};")
    spark.sql(f"""CREATE TABLE IF NOT EXISTS {self.database}.{self.table_name} USING DELTA LOCATION '{self.sink}';""")
    dbutils.fs.rm(self.source)
  
  def register_data(self):
    self._acquire_raw()
    self._transfer_local()
    data = self._read_source()
    self._write_source(data)
    self._create_table()
    
  def get_data(self):
    return spark.table(f"{self.database}.{self.table_name}")

Configuration for data ingestion and registration, followed by execution of the ingest.

In [0]:
DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
LOCAL_FILE = "file:/databricks/driver/adult.data"
SOURCE = "dbfs:/home/benjamin.wilson@databricks.com/demo/adultdata/adult.csv" # Change this to your local workspace pathing if you want to run this
SINK = "/home/benjamin.wilson@databricks.com/demo/adult" # Change this to your local workspace pathing if you want to run this
DATABASE = "ben_demo" # Change this if you want to run this
TABLE_NAME = "adult_classification"

ADULT_SCHEMA = StructType([
  StructField('age', IntegerType()),
  StructField('working_class', StringType()),
  StructField('final_weight', DoubleType()),
  StructField('education_level', StringType()),
  StructField('education_years', DoubleType()),
  StructField('marital_status', StringType()),
  StructField('occupation', StringType()),
  StructField('relationship_type', StringType()),
  StructField('race', StringType()),
  StructField('gender', StringType()),
  StructField('capital_gain', DoubleType()),
  StructField('capital_loss', DoubleType()),
  StructField('hours_worked_per_week', DoubleType()),
  StructField('native_country', StringType()),
  StructField('income', StringType())
])

data_handler = DataIngest(DATA_URL, LOCAL_FILE, SOURCE, SINK, ADULT_SCHEMA, DATABASE, TABLE_NAME)
data_handler.register_data()

Let's take a look at what this raw data looks like

In [0]:
raw_data = data_handler.get_data()
display(raw_data)

age,working_class,final_weight,education_level,education_years,marital_status,occupation,relationship_type,race,gender,capital_gain,capital_loss,hours_worked_per_week,native_country,income
50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
37,Private,284582.0,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K
49,Private,160187.0,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.0,0.0,16.0,Jamaica,<=50K
52,Self-emp-not-inc,209642.0,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,>50K
31,Private,45781.0,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,>50K
42,Private,159449.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.0,0.0,40.0,United-States,>50K
37,Private,280464.0,Some-college,10.0,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0.0,0.0,80.0,United-States,>50K


This isn't particularly useable. The label for this training data is a string of " >50K" or " <=50K". In fact, every single string type value that is in the data set is preceded by a space. While this may not make much difference for encoding algorithms, it certainly will give some problems for analytics use cases that need to use this data. 
There are also a few features that we'd like to define engineered versions of for different use cases. Let's define all of these operations as a series of functions.

In [0]:
@dataclass
class SchemaTypes:
  string_cols: List[str]
  non_string_cols: List[str]

def get_col_types(df):
  schema = df.schema
  strings = [x.name for x in schema if x.dataType == StringType()]
  non_strings = [x for x in schema.names if x not in strings]
  return SchemaTypes(strings, non_strings)

def clean_messy_strings(df):
  cols = get_col_types(df)
  return df.select(*cols.non_string_cols, *[F.regexp_replace(F.col(x), " ", "").alias(x) for x in cols.string_cols])

def fill_missing(df):
  cols = get_col_types(df)
  return df.select(*cols.non_string_cols, *[when(F.col(x) == "?", "Unknown").otherwise(F.col(x)).alias(x) for x in cols.string_cols])

def convert_label(df, label, true_condition_string):
  return df.withColumn(label, when(F.col(label) == true_condition_string, 1).otherwise(0))

def generate_features(df, id_augment):
  overtime = df.withColumn("overtime", when(F.col("hours_worked_per_week") > 40, 1).otherwise(0))
  net_pos = overtime.withColumn("gains", when(F.col("capital_gain") > F.col("capital_loss"), 1).otherwise(0))
  high_edu = net_pos.withColumn("highly_educated", when(F.col("education_years") >= 16, 2).when(F.col("education_years") > 12, 1).otherwise(0))
  gender = high_edu.withColumn("gender_key", when(F.col("gender") == "Female", 1).otherwise(0))
  keys = gender.withColumn("id", F.monotonically_increasing_id() + F.lit(id_augment))
  return keys

Now let's build the final dataset with a controller function that will clean, augment, and materialize all of our operations.

In [0]:
def data_augmentation(df, label, label_true_condition, id_augment=0):
  clean_strings = clean_messy_strings(df)
  missing_filled = fill_missing(clean_strings)
  corrected_label = convert_label(missing_filled, label, label_true_condition)
  additional_features = generate_features(corrected_label, id_augment)
  return additional_features

And let's see what those changes look like.

In [0]:
augmented = data_augmentation(raw_data, "income", ">50K")
display(augmented)

age,final_weight,education_years,capital_gain,capital_loss,hours_worked_per_week,working_class,education_level,marital_status,occupation,relationship_type,race,gender,native_country,income,overtime,gains,highly_educated,gender_key,id
50,83311.0,13.0,0.0,0.0,13.0,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0,0,0,1,0,0
38,215646.0,9.0,0.0,0.0,40.0,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,0,0,0,0,0,1
53,234721.0,7.0,0.0,0.0,40.0,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,0,0,0,0,0,2
28,338409.0,13.0,0.0,0.0,40.0,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,0,0,0,1,1,3
37,284582.0,14.0,0.0,0.0,40.0,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,0,0,0,1,1,4
49,160187.0,5.0,0.0,0.0,16.0,Private,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,Jamaica,0,0,0,0,1,5
52,209642.0,9.0,0.0,0.0,45.0,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1,1,0,0,0,6
31,45781.0,14.0,14084.0,0.0,50.0,Private,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,United-States,1,1,1,1,1,7
42,159449.0,13.0,5178.0,0.0,40.0,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1,0,1,1,0,8
37,280464.0,10.0,0.0,0.0,80.0,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,United-States,1,1,0,0,0,9


Now, to set about defining a materialized version of these features so that models, analysts, and we can all access the exact same processing logic from a single simple command.

Instantiate the feature store client handler so that we can register the table in the feature store.

In [0]:
fs = feature_store.FeatureStoreClient()

Define the table and the parameters of it. <br>
##### NOTE
This is a one-time irreversible action. Feature Store tables cannot be removed, but they can be updated, modified, and added to. Make sure that the naming of this table and the primary key definition is correct before committing a registration action.

In [0]:
FEATURE_TABLE = "ben_demo.adult_data_features_ver2"
FEATURE_KEYS = ["id"]
FEATURE_PARTITION = "gender"

Create and register the logic for processing ingestion to the feature store table. NOTE: additional information can be added to the table description within the GUI later on if needed.

In [0]:
fs.create_feature_table(
  name=FEATURE_TABLE,
  keys=["id"],
  features_df=data_augmentation(raw_data, "income", ">50K"),
  partition_columns=FEATURE_PARTITION,
  description="Adult Salary Data. Raw Features."
)

2021/07/01 19:46:54 INFO databricks.feature_store.utils.spark_listener: Actively listening to data sources.
Out[84]: <FeatureTable: >

Ok, so some things just happened. Let's see what happens when we query that table. (Scroll to the right to see those new feature columns that we created above)

In [0]:
display(spark.table(FEATURE_TABLE))

age,final_weight,education_years,capital_gain,capital_loss,hours_worked_per_week,working_class,education_level,marital_status,occupation,relationship_type,race,gender,native_country,income,overtime,gains,highly_educated,gender_key,id
28,338409.0,13.0,0.0,0.0,40.0,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,0,0,0,1,1,3
37,284582.0,14.0,0.0,0.0,40.0,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,United-States,0,0,0,1,1,4
49,160187.0,5.0,0.0,0.0,16.0,Private,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,Jamaica,0,0,0,0,1,5
31,45781.0,14.0,14084.0,0.0,50.0,Private,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,United-States,1,1,1,1,1,7
23,122272.0,13.0,0.0,0.0,30.0,Private,Bachelors,Never-married,Adm-clerical,Own-child,White,Female,United-States,0,0,0,1,1,11
43,292175.0,14.0,0.0,0.0,45.0,Self-emp-not-inc,Masters,Divorced,Exec-managerial,Unmarried,White,Female,United-States,1,1,0,1,1,18
54,302146.0,9.0,0.0,0.0,20.0,Private,HS-grad,Separated,Other-service,Unmarried,Black,Female,United-States,0,0,0,0,1,20
59,109015.0,9.0,0.0,0.0,40.0,Private,HS-grad,Divorced,Tech-support,Unmarried,White,Female,United-States,0,0,0,0,1,23
19,544091.0,9.0,0.0,0.0,25.0,Private,HS-grad,Married-AF-spouse,Adm-clerical,Wife,White,Female,United-States,0,0,0,0,1,36
49,94638.0,9.0,0.0,0.0,40.0,Private,HS-grad,Separated,Adm-clerical,Unmarried,White,Female,United-States,0,0,0,0,1,42


What if we get new data coming in? Let's check the row count of our table first...

In [0]:
table_counts = spark.table(FEATURE_TABLE).count()
table_counts

Out[87]: 32560

Simulate loading new data (we're just reusing already existing data from the same source dataset here)

In [0]:
new_data = data_handler.get_data().limit(100)
processed_new_data = data_augmentation(new_data, "income", ">50K", table_counts)

fs.write_table(
  name=FEATURE_TABLE,
  df=processed_new_data,
  mode='merge'
)

2021/07/01 19:48:08 INFO databricks.feature_store.utils.spark_listener: Actively listening to data sources.

...and now let's see the row count after we added this new data.

In [0]:
spark.table(FEATURE_TABLE).count()

Out[89]: 32660

Cool. We added 100 duplicate rows with new primary keys (don't do this in real life). We can also stream data into this table with our embedded feature engineering code processing against the stream. The only change for this is to use a `spark.readStream` from the source and the feature store client `.write_table()` signature is the same. But what about using this feature for ML uses?

We can create a training data set from this table.

In [0]:
def generate_lookup(table, feature, key):
  return FeatureLookup(
    table_name=table,
    feature_name=feature,
    lookup_key=key
  )

Build a very simplistic model that uses the feature store table as its source for training and validation.

In [0]:
EXPERIMENT_TITLE = "Adult_Catboost_2"
MODEL_TYPE = "adult_catboost_classifier"
EXPERIMENT_NAME = f"/Users/benjamin.wilson@databricks.com/Book/{EXPERIMENT_TITLE}"
mlflow.set_experiment(EXPERIMENT_NAME)
with mlflow.start_run():
  
  TEST_SIZE = 0.15
  
  features = ["overtime", "gains", "highly_educated", "age", "education_years", "hours_worked_per_week", "gender_key"]
  lookups = [generate_lookup(FEATURE_TABLE, x, "id") for x in features]
  
  training_df = spark.table(FEATURE_TABLE).select("id", "income")
  
  training_data = fs.create_training_set(
    df=training_df,
    feature_lookups=lookups,
    label="income",
    exclude_columns=['id', 'final_weight', 'capital_gain', 'capital_loss']
  )

  train_df = training_data.load_df().toPandas()
  
  X = train_df.drop(['income'], axis=1)
  y = train_df.income
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42, stratify=y)
  
  model = CatBoostClassifier(
    iterations=10000,
    boosting_type="Ordered",
    learning_rate=0.00001,
    auto_class_weights="SqrtBalanced",
    leaf_estimation_method="Newton",
    custom_loss=[cb_metrics.AUC()],
    random_seed=42,
    logging_level="Silent"
  ).fit(X_train, y_train, eval_set=(X_test, y_test), logging_level="Verbose")
  signature = mlflow.models.signature.infer_signature(X_train, model.predict(X_train))
  
  pred = model.predict(X_test)
  fpr, tpr, thresholds = metrics.roc_curve(y_test, pred, pos_label=1)
  auc = metrics.auc(fpr, tpr)
  accuracy = metrics.accuracy_score(y_test, pred)
  importances = dict(zip(model.feature_names_, model.feature_importances_))
  
  mlflow.log_dict(importances, "feature_importances.json")
  mlflow.log_metric("auc", auc)
  mlflow.log_metric("accuracy", accuracy)
  mlflow.log_param("split_size", TEST_SIZE)
  mlflow.log_params(model.get_params())
  mlflow.set_tag("model_info", "feature store test")
  mlflow.catboost.log_model(model, MODEL_TYPE, signature=signature)
  
  # now let's register this model to the feature store
  fs.log_model(
    model,
    MODEL_TYPE,
    flavor=mlflow.catboost,
    training_set=training_data,
    registered_model_name=MODEL_TYPE
  )
  

/local_disk0/.ephemeral_nfs/envs/pythonEnv-ea16626f-9f41-4515-adb7-b6fcaa4b940f/lib/python3.8/site-packages/mlflow/models/signature.py:127: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
 inputs = _infer_schema(model_input)
Registered model 'adult_catboost_classifier' already exists. Creating a new version of this model...
2021/07/01 21:01:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: adult_catboost_classifier, version 19
Created version '19' of model 'adult_catboost_classifier'.

Now that we have the model logged to feature store, let's get the latest version from the experiment and use it.

In [0]:
client = MlflowClient()
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id
run_id = mlflow.search_runs(experiment_id, order_by=["start_time DESC"]).head(1)["run_id"].values[0]

With the feature store registration associated with the MLflow model, we don't have to specify any data loading and processing to happen other than a point to the raw data that features will be calculated from. We can do batch predictions simply by accessing the feature store instance, providing the run_id and the model's name (MODEL_TYPE below) with the raw data specified as the 2nd argument.

In [0]:
feature_store_predictions = fs.score_batch(f"runs:/{run_id}/{MODEL_TYPE}", spark.table(FEATURE_TABLE))

In [0]:
display(feature_store_predictions)

age,final_weight,education_years,capital_gain,capital_loss,hours_worked_per_week,working_class,education_level,marital_status,occupation,relationship_type,race,gender,native_country,income,overtime,gains,highly_educated,id,prediction
49,160187.0,5.0,0.0,0.0,16.0,Private,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,Jamaica,0,0,0,0,5,0.0
31,45781.0,14.0,14084.0,0.0,50.0,Private,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,United-States,1,1,1,1,7,1.0
18,226956.0,9.0,0.0,0.0,30.0,Private,HS-grad,Never-married,Other-service,Own-child,White,Female,Unknown,0,0,0,0,50,0.0
47,51835.0,15.0,0.0,1902.0,60.0,Private,Prof-school,Married-civ-spouse,Prof-specialty,Wife,White,Female,Honduras,1,1,0,1,51,1.0
28,183175.0,10.0,0.0,0.0,40.0,Private,Some-college,Divorced,Adm-clerical,Not-in-family,White,Female,United-States,0,0,0,0,65,0.0
18,309634.0,7.0,0.0,0.0,22.0,Private,11th,Never-married,Other-service,Own-child,White,Female,United-States,0,0,0,0,77,0.0
53,346253.0,9.0,0.0,0.0,35.0,Private,HS-grad,Divorced,Sales,Own-child,White,Female,United-States,0,0,0,0,84,0.0
43,410867.0,16.0,0.0,0.0,50.0,Federal-gov,Doctorate,Never-married,Prof-specialty,Not-in-family,White,Female,United-States,1,1,0,2,88,1.0
31,189265.0,9.0,0.0,0.0,40.0,Local-gov,HS-grad,Never-married,Adm-clerical,Not-in-family,White,Female,United-States,0,0,0,0,107,0.0
28,377869.0,10.0,4064.0,0.0,25.0,Private,Some-college,Married-civ-spouse,Sales,Wife,White,Female,United-States,0,0,1,0,112,0.0


In [0]:
importances = dict(zip(model.feature_names_, model.feature_importances_))
importances

Out[97]: {'overtime': 2.7811379714994744,
 'gains': 13.620240267109127,
 'highly_educated': 12.514164012263587,
 'age': 37.796819032703574,
 'education_years': 13.023024206894078,
 'hours_worked_per_week': 5.979387010850218,
 'gender_key': 14.285227498679944}